# <img src="https://github.com/JuliaLang/julia-logo-graphics/raw/master/images/julia-logo-color.png" height="100" /> Notebook for Chamber.jl

- Gihub: [Chamber.jl](https://github.com/brown-ccv/Chamber.jl)
- This notebook is adapted from https://github.com/ageron/julia_notebooks.
- About Google Colaboratory: [FAQ](https://research.google.com/colaboratory/faq.html)

## Instructions
1. Work on a copy of this notebook: _File_ > _Save a copy in Drive_ (you will need a Google account). Alternatively, you can download the notebook using _File_ > _Download .ipynb_, then upload it to [Colab](https://colab.research.google.com/).
2. Execute the following cell (click on it and press Ctrl+Enter) to install Julia, IJulia and other packages.
3. Reload this page (press Ctrl+R, or ⌘+R, or the F5 key) and continue to the next section.

_Notes_:
* If your Colab Runtime gets reset (e.g., due to inactivity), repeat steps 2 and 3.
* It may need 15-20 minutes.

In [ ]:
%%shell
set -e

#---------------------------------------------------#
JULIA_VERSION="1.8.2"
JULIA_PACKAGES="IJulia Chamber"
JULIA_NUM_THREADS=2
#---------------------------------------------------#

if [ -z `which julia` ]; then
  # Install Julia
  JULIA_VER=`cut -d '.' -f -2 <<< "$JULIA_VERSION"`
  echo "Installing Julia $JULIA_VERSION on the current Colab Runtime..."
  BASE_URL="https://julialang-s3.julialang.org/bin/linux/x64"
  URL="$BASE_URL/$JULIA_VER/julia-$JULIA_VERSION-linux-x86_64.tar.gz"
  wget -nv $URL -O /tmp/julia.tar.gz # -nv means "not verbose"
  tar -x -f /tmp/julia.tar.gz -C /usr/local --strip-components 1
  rm /tmp/julia.tar.gz

  # Install Packages
  for PKG in `echo $JULIA_PACKAGES`; do
    echo "Installing Julia package $PKG..."
    julia -e 'using Pkg; pkg"add '$PKG'; precompile;"' &> /dev/null
  done

  # Install kernel and rename it to "julia"
  echo "Installing IJulia kernel..."
  julia -e 'using IJulia; IJulia.installkernel("julia", env=Dict(
      "JULIA_NUM_THREADS"=>"'"$JULIA_NUM_THREADS"'"))'
  KERNEL_DIR=`julia -e "using IJulia; print(IJulia.kerneldir())"`
  KERNEL_NAME=`ls -d "$KERNEL_DIR"/julia*`
  mv -f $KERNEL_NAME "$KERNEL_DIR"/julia  

  echo ''
  echo "Successfully installed `julia -v`!"
  echo "Please reload this page (press Ctrl+R, ⌘+R, or the F5 key) then"
  echo "jump to the 'Checking the Installation' section."
fi

Installing Julia 1.8.2 on the current Colab Runtime...
2023-05-08 14:26:37 URL:https://storage.googleapis.com/julialang2/bin/linux/x64/1.8/julia-1.8.2-linux-x86_64.tar.gz [135859273/135859273] -> "/tmp/julia.tar.gz" [1]
Installing Julia package IJulia...
Installing Julia package Chamber...
Installing IJulia kernel...
[ Info: Installing julia kernelspec in /root/.local/share/jupyter/kernels/julia-1.8

Please reload this page (press Ctrl+R, ⌘+R, or the F5 key) then
jump to the 'Checking the Installation' section.


# Checking the Installation
The `versioninfo()` function should print your Julia version and some other info about the system:

In [1]:
versioninfo()

Julia Version 1.8.2
Commit 36034abf260 (2022-09-29 15:21 UTC)
Platform Info:
  OS: Linux (x86_64-linux-gnu)
  CPU: 2 × Intel(R) Xeon(R) CPU @ 2.20GHz
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-13.0.1 (ORCJIT, broadwell)
  Threads: 2 on 2 virtual cores
Environment:
  LD_LIBRARY_PATH = /usr/local/nvidia/lib:/usr/local/nvidia/lib64
  JULIA_NUM_THREADS = 2


Check Julia package

In [2]:
using Pkg
Pkg.status()

Status `~/.julia/environments/v1.8/Project.toml`
  [4ddda095] Chamber v0.1.0
  [7073ff75] IJulia v1.24.0


## Try Chamber.jl

### Using Chamber

In [7]:
using Chamber
@doc chamber

```
chamber(composition::Union{Silicic,Mafic}, end_time::Float64, log_volume_km3::Float64, InitialConc_H2O::Float64, InitialConc_CO2::Float64, log_vfr::Float64, depth::Float64, output_dirname::String)
```

Simulate the eruption of a volcano using a model for the frequency of eruptions of upper crustal magma chambers based on Degruyter and Huber (2014).

## Arguments

  * `composition`: The magma composition. Use `Silicic()` for rhyolite composition (arc magma) or `Mafic()` for basalt composition (rift).
  * `end_time`: Maximum magma chamber evolution duration in seconds.
  * `log_volume_km3`: The initial volume of the chamber in logarithmic scale. The actual initial chamber volume is calculated as 10^(`log_volume_km3`) in km³.
  * `InitialConc_H2O`: The initial weight fraction of water in the magma (exsolved + dissolved).
  * `InitialConc_CO2`: The initial weight fraction of CO₂ in the magma (exsolved + dissolved).
  * `log_vfr`: Magma recharge rate in km³/yr calculated as 10^(`log_vfr`).
  * `depth`: Depth of the magma chamber in meters.
  * `output_dirname`(optional): Name of the output directory. Defaults to current timestamp.

## Returns

A `DataFrame` containing the solution with columns:

  * `time`: Simulation timestamps in seconds.
  * `P+dP`: Pressure in Pa.
  * `T`: Temperature in K.
  * `eps_g`: Gas volume fraction.
  * `V`: Volume of the magma chamber in m³.
  * `rho_m`: Density of the melt in kg/m³.
  * `rho_x`: Density of magma crystal in kg/m³.
  * `X_CO2`: Mole fraction of CO2 in the gas.
  * `total_mass`: Total mass of magma chamber in kg.
  * `total_mass_H2O`: Total mass of water in the magma in kg.
  * `total_mass_CO2`: Total mass of CO₂ in the magma in kg.

## Outputs

A directory named after `output_dirname` or the default value, containing the following files:

  * `out.csv`: a CSV file containing the solution columns listed above.
  * `eruptions.csv`, A CSV file containing the datas of eruptions with the following columns: time*of*eruption (sec), duration*of*eruption (sec), mass*erupted (kg) and volume*erupted (km³).
  * Figures for P+dP(t), T(t), eps*g(t), V(t), X*CO2(t), total_mass(t).

## References

  * W. Degruyter and C. Huber (2014). A model for eruption frequency of upper crustal silicic magma chambers. Earth Planet. Sci. Lett. https://doi.org/10.1016/j.epsl.2014.06.047

## Examples

```
# Run a simulation with silicic magma chamber
julia> composition = Silicic()
julia> end_time = 3e9
julia> log_volume_km3 = 0.2
julia> InitialConc_H2O = 0.04
julia> InitialConc_CO2 = 0.001
julia> log_vfr = -3.3
julia> depth = 8e3

julia> chamber(composition, end_time, log_volume_km3, InitialConc_H2O, InitialConc_CO2, log_vfr, depth)

# Run a simulation with mafic magma chamber, with custom directory name "MyDirname"
julia> composition = Mafic()
julia> end_time = 3e9
julia> log_volume_km3 = 0.2
julia> InitialConc_H2O = 0.01
julia> InitialConc_CO2 = 0.001
julia> log_vfr = -3.3
julia> depth = 8e3
julia> output_dirname = "MyDirname"

julia> chamber(composition, end_time, log_volume_km3, InitialConc_H2O, InitialConc_CO2, log_vfr, depth, output_dirname)
```


### Example 1: Silicic case
* Please use float data type for the values of `end_time`, `log_volume_km3`, `InitialConc_H2O`, `InitialConc_CO2`, `log_vfr` and `depth`.

In [5]:
composition = Silicic()
end_time = 3e9
log_volume_km3 = 0.2
InitialConc_H2O = 0.04
InitialConc_CO2 = 0.001
log_vfr = -3.3
depth = 8e3
dataframe = chamber(composition, end_time, log_volume_km3, InitialConc_H2O, InitialConc_CO2, log_vfr, depth)

Output path: /content/202305081453
number_of_data: 470


Row,time,P+dP,T,eps_g,V,rho_m,rho_x,X_CO2,total_mass,total_mass_H2O,total_mass_CO2
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,0.0,2.1582e8,1046.71,0.00558159,1.58489e9,2400.0,2600.0,0.346043,3.83394e12,1.54039e11,3.85098e9
2,100000.0,2.15824e8,1046.71,0.00558131,1.58489e9,2400.0,2600.0,0.34605,3.83395e12,1.54039e11,3.85099e9
3,7.62058e5,2.15848e8,1046.71,0.0055795,1.5849e9,2400.01,2600.01,0.346101,3.83397e12,1.5404e11,3.85101e9
4,2.83015e6,2.15924e8,1046.71,0.00557383,1.58491e9,2400.03,2600.03,0.346259,3.83405e12,1.54044e11,3.85109e9
5,4.89825e6,2.16e8,1046.71,0.00556818,1.58492e9,2400.04,2600.05,0.346418,3.83413e12,1.54047e11,3.85117e9
6,6.96635e6,2.16077e8,1046.7,0.00556253,1.58493e9,2400.06,2600.07,0.346576,3.83421e12,1.5405e11,3.85125e9
7,9.03444e6,2.16153e8,1046.7,0.00555688,1.58495e9,2400.08,2600.09,0.346733,3.83429e12,1.54053e11,3.85133e9
8,1.33676e7,2.16312e8,1046.7,0.00554509,1.58497e9,2400.12,2600.13,0.347064,3.83445e12,1.5406e11,3.8515e9
9,2.08499e7,2.16588e8,1046.69,0.0055248,1.58502e9,2400.18,2600.2,0.347633,3.83474e12,1.54071e11,3.85178e9


### Example 2: Mafic case

In [6]:
composition = Mafic()
end_time = 3e9
log_volume_km3 = 0.2
InitialConc_H2O = 0.01
InitialConc_CO2 = 0.001
log_vfr = -3.3
depth = 8e3
dataframe = chamber(composition, end_time, log_volume_km3, InitialConc_H2O, InitialConc_CO2, log_vfr, depth)

Output path: /content/202305081504
number_of_data: 458


Row,time,P+dP,T,eps_g,V,rho_m,rho_x,X_CO2,total_mass,total_mass_H2O,total_mass_CO2
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,0.0,2.1582e8,1405.74,0.00139209,1.58489e9,2420.0,2900.0,0.859079,3.94439e12,3.94903e10,3.94903e9
2,100000.0,2.15823e8,1405.74,0.00139197,1.58489e9,2420.0,2900.0,0.859081,3.9444e12,3.94903e10,3.94903e9
3,6.31904e5,2.1584e8,1405.74,0.00139134,1.5849e9,2420.0,2900.01,0.859088,3.94442e12,3.94905e10,3.94905e9
4,2.29355e6,2.15892e8,1405.74,0.00138939,1.5849e9,2420.02,2900.02,0.859112,3.94448e12,3.94911e10,3.94911e9
5,3.95521e6,2.15944e8,1405.74,0.00138744,1.58491e9,2420.03,2900.04,0.859136,3.94455e12,3.94918e10,3.94918e9
6,5.61686e6,2.15996e8,1405.74,0.00138549,1.58492e9,2420.04,2900.05,0.859159,3.94461e12,3.94924e10,3.94924e9
7,7.27851e6,2.16048e8,1405.74,0.00138354,1.58493e9,2420.06,2900.07,0.859183,3.94467e12,3.94931e10,3.94931e9
8,1.07801e7,2.16158e8,1405.74,0.00137944,1.58495e9,2420.08,2900.1,0.859233,3.94481e12,3.94944e10,3.94944e9
9,1.72312e7,2.1636e8,1405.75,0.00137191,1.58498e9,2420.13,2900.16,0.859324,3.94506e12,3.94969e10,3.94969e9


### Example 3: Silicic saturated case

In [ ]:
composition = Silicic()
end_time = 3e9
log_volume_km3 = 0.2
InitialConc_H2O = 0.05
InitialConc_CO2 = 0.0001
log_vfr = -3.3
depth = 8e3
dataframe = chamber(composition, end_time, log_volume_km3, InitialConc_H2O, InitialConc_CO2, log_vfr, depth)

### Example 4: Mafic saturated case

In [ ]:
composition = Mafic()
end_time = 6e9
log_volume_km3 = 0.2
InitialConc_H2O = 0.01
InitialConc_CO2 = 0.0008
log_vfr = -3.3
depth = 8e3
dataframe = chamber(composition, end_time, log_volume_km3, InitialConc_H2O, InitialConc_CO2, log_vfr, depth)